# 🏃 Half Marathon Finish Time Predictor — Training Pipeline

Pipeline trenowania modelu ML do szacowania czasu ukończenia półmaratonu.

**Features wejściowe (dostępne przed startem):**
- Płeć (M/K)
- Wiek (obliczany z rocznika)
- Tempo na 5 km (min/km)

**Target:** Czas ukończenia biegu (w sekundach)

In [ ]:
# ── Instalacja zależności ──────────────────────────────────────────
# %pip install boto3 scikit-learn pandas joblib python-dotenv

In [ ]:
import os
import io
import joblib
import boto3
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from datetime import datetime

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, r2_score

load_dotenv()
print('✅ Biblioteki załadowane')

## 1. Konfiguracja Digital Ocean Spaces

In [ ]:
# Uzupełnij w pliku .env lub ustaw zmienne środowiskowe
DO_SPACES_KEY    = os.getenv('DO_SPACES_KEY')
DO_SPACES_SECRET = os.getenv('DO_SPACES_SECRET')
DO_SPACES_REGION = os.getenv('DO_SPACES_REGION', 'fra1')   # np. fra1, nyc3
DO_SPACES_BUCKET = os.getenv('DO_SPACES_BUCKET', 'halfmarathon-ml')
DO_SPACES_ENDPOINT = f'https://{DO_SPACES_REGION}.digitaloceanspaces.com'

s3 = boto3.client(
    's3',
    region_name=DO_SPACES_REGION,
    endpoint_url=DO_SPACES_ENDPOINT,
    aws_access_key_id=DO_SPACES_KEY,
    aws_secret_access_key=DO_SPACES_SECRET,
)
print(f'✅ Połączono z DO Spaces: {DO_SPACES_BUCKET} @ {DO_SPACES_REGION}')

## 2. Wczytanie danych z DO Spaces

In [ ]:
def read_csv_from_spaces(key: str) -> pd.DataFrame:
    """Pobiera plik CSV z Digital Ocean Spaces i zwraca DataFrame."""
    response = s3.get_object(Bucket=DO_SPACES_BUCKET, Key=key)
    content  = response['Body'].read()
    return pd.read_csv(io.BytesIO(content), sep=';')

df_2023 = read_csv_from_spaces('data/halfmarathon_wroclaw_2023__final.csv')
df_2024 = read_csv_from_spaces('data/halfmarathon_wroclaw_2024__final.csv')

print(f'📁 2023: {df_2023.shape}  |  2024: {df_2024.shape}')

## 3. Czyszczenie i przygotowanie danych

In [ ]:
def convert_time_to_seconds(t):
    """HH:MM:SS → sekundy. Zwraca None dla DNS/DNF."""
    if pd.isnull(t) or str(t).strip() in ('DNS', 'DNF', ''):
        return None
    parts = str(t).split(':')
    return int(parts[0]) * 3600 + int(parts[1]) * 60 + int(parts[2])


def clean_dataframe(df: pd.DataFrame, race_year: int) -> pd.DataFrame:
    df = df.copy()

    # ── Czas ukończenia ────────────────────────────────────────────
    df['Czas_s'] = df['Czas'].apply(convert_time_to_seconds)

    # ── Tempo na 5 km ──────────────────────────────────────────────
    # Kolumna '5 km Tempo' już jest w min/km jako float
    df['tempo_5km'] = pd.to_numeric(df['5 km Tempo'], errors='coerce')

    # ── Płeć ───────────────────────────────────────────────────────
    df['plec'] = df['Płeć'].map({'M': 0, 'K': 1})

    # ── Wiek ───────────────────────────────────────────────────────
    df['rocznik'] = pd.to_numeric(df['Rocznik'], errors='coerce')
    df['wiek'] = race_year - df['rocznik']

    # ── Filtrowanie ────────────────────────────────────────────────
    df = df.dropna(subset=['Czas_s', 'tempo_5km', 'plec', 'wiek'])

    # Usuwamy wartości odstające (ukończenie między 55 min a 4h)
    df = df[(df['Czas_s'] >= 3300) & (df['Czas_s'] <= 14400)]
    # Wiek rozsądny
    df = df[(df['wiek'] >= 15) & (df['wiek'] <= 90)]
    # Tempo 5km rozsądne (2.5 – 15 min/km)
    df = df[(df['tempo_5km'] >= 2.5) & (df['tempo_5km'] <= 15)]

    return df[['plec', 'wiek', 'tempo_5km', 'Czas_s']]


df_2023_clean = clean_dataframe(df_2023, 2023)
df_2024_clean = clean_dataframe(df_2024, 2024)

df_all = pd.concat([df_2023_clean, df_2024_clean], ignore_index=True)

print(f'📊 Łącznie po czyszczeniu: {len(df_all)} rekordów')
print(df_all.describe())

## 4. Feature Selection — analiza korelacji

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

corr = df_all.corr()
print('Korelacja z czasem ukończenia (Czas_s):')
print(corr['Czas_s'].sort_values(ascending=False))

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for ax, feat in zip(axes, ['tempo_5km', 'wiek', 'plec']):
    ax.scatter(df_all[feat], df_all['Czas_s'] / 60, alpha=0.15, s=5)
    ax.set_xlabel(feat)
    ax.set_ylabel('Czas [min]')
    ax.set_title(f'{feat} vs Czas')
plt.tight_layout()
plt.show()

# Wnioski: tempo_5km jest najsilniejszym predyktorem (~0.97 korelacja)

## 5. Trenowanie modelu

In [ ]:
FEATURES = ['plec', 'wiek', 'tempo_5km']
TARGET   = 'Czas_s'

X = df_all[FEATURES]
y = df_all[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    random_state=42,
)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mae_s  = mean_absolute_error(y_test, y_pred)
r2     = r2_score(y_test, y_pred)

print(f'📈 MAE  : {mae_s:.0f} s  ({mae_s/60:.1f} min)')
print(f'📈 R²   : {r2:.4f}')

# Cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error')
print(f'📈 CV MAE (5-fold): {-cv_scores.mean():.0f} ± {cv_scores.std():.0f} s')

In [ ]:
# Feature importance
importances = pd.Series(model.feature_importances_, index=FEATURES).sort_values(ascending=False)
print('\nFeature Importance:')
print(importances)

## 6. Zapis modelu — lokalnie i do DO Spaces

In [ ]:
import json

timestamp   = datetime.now().strftime('%Y%m%d_%H%M%S')
model_fname = f'model_{timestamp}.joblib'
meta_fname  = f'model_{timestamp}_meta.json'

# ── Zapis lokalny ─────────────────────────────────────────────────
os.makedirs('models', exist_ok=True)
joblib.dump(model, f'models/{model_fname}')

# Zapisz też jako 'latest' dla aplikacji
joblib.dump(model, 'models/model_latest.joblib')

meta = {
    'timestamp' : timestamp,
    'features'  : FEATURES,
    'mae_s'     : round(mae_s, 2),
    'r2'        : round(r2, 4),
    'n_samples' : len(df_all),
    'model_type': type(model).__name__,
}
with open(f'models/{meta_fname}', 'w') as f:
    json.dump(meta, f, indent=2)
with open('models/model_latest_meta.json', 'w') as f:
    json.dump(meta, f, indent=2)

print(f'✅ Model zapisany lokalnie: models/{model_fname}')

# ── Upload do DO Spaces ───────────────────────────────────────────
for local, remote in [
    (f'models/{model_fname}',    f'models/{model_fname}'),
    ('models/model_latest.joblib',       'models/model_latest.joblib'),
    (f'models/{meta_fname}',     f'models/{meta_fname}'),
    ('models/model_latest_meta.json',    'models/model_latest_meta.json'),
]:
    s3.upload_file(local, DO_SPACES_BUCKET, remote)
    print(f'☁️  Wysłano: {remote}')

print('\n🏁 Pipeline zakończony!')
print(json.dumps(meta, indent=2))